#Lecture 6.1 - Factorization Machines (FM)

##What you'll learn from this lecture
1.  What is a factorization machine
2.  How FM extends linear methods
3.  How FM solution space includes
    a.  linear regression
    b.  penalized regression
    c.  polynomial basis expansion
    d.  SVM
    e.  Matrix factor recommenders

##Readings
http://tech.adroll.com/blog/data-science/2015/08/25/factorization-machines.html#references - great one-page summary

http://www.ismll.uni-hildesheim.de/pub/pdfs/Rendle2010FM.pdf - Rendel's original paper introducing factorization machines

https://en.wikipedia.org/wiki/Polynomial_kernel - Description of polynomial kernel, relationship to feature space expansion.

https://en.wikipedia.org/wiki/Kernel_method - Kernel method

https://en.wikipedia.org/wiki/Polynomial_kernel - Polynomial kernels

https://en.wikipedia.org/wiki/Loss_functions_for_classification#Logistic_loss - Description and comparison between different loss functions for classification

https://en.wikipedia.org/wiki/Hinge_loss - Description of hinge loss function

##What is a factorization machine?
The usual machine learning problem presents us with two things - attributes and labels.  Machine learning algorithms build relationships betwen these two.  Suppose the label are called y and the vector of attributes is called x.  Machine learning algorithms use training data to assemble a function f() so that $y \approx f(x)$.  The squiggly equals sign denote approximation.  The differences between machine learning algorithms are how narrowly constrained are the functions they build, the way they quantify the differences from equality between y and f(x) and the process they use to build (or train) the function f().  Some algorithms can build extremely complicated functions and are called non-parametric because they're not simply twiddling a couple of parameters.  By now you're familiar with ensembles of trees and neural networks.  Those are both non-parametric techniques.  On the other hand some algorithms place constraints on the nature of the approximating function.  One extremely important class is linear methods.  For linear methods, the approximations all look something like
$$y \approx w_0 + \sum_{i=1}^N w_i x_i \triangleq \hat{y}(\textbf x)$$


By now you're familiar with a number of linear algorithms - linear regression, penalized linear regression, support vector machines (SVM), logistic regression or linear discriminant analysis.  The differences between these again have to do with how they quantify error, how they deal with overfitting and so forth.  You can arrange these difference along two different dimension.  One dimension is whether the problem is a regression or classification problem.  The second is the nature of the scoring function that is minimized.  Rendle's paper goes through the details of expressing the gradient of the FM prediction equation.  That's important because extracting that gradient can be computationally expensive and one of the paper's key contributions is working out a way to arrange the gradient of the factor representation that only depends linearly on the number of rows in the feature matrix plus the number of attributes.  The rest of the gradient calculation depends on the specifics of the cost function.  The user gets to choose the loss function and workout the dependence of the gradient of the accumulated losses on the gradient of the prediction.  

##Questions
Suppose that your problem comes with a data matrix $X$ associated labels $Y$ and a prediction function $f(x, w)$ where lower case $x$ is a row from upper case $X$.  The function $f$ exposes $w$ a vector of parameters that you get to chose to minimize a loss function.  Show how the accumulated loss depends on the gradient of $\nabla_wf$ is related to the gradient of the accumulated loss for the following examples (the subscript $w$ is to emphasize that the gradient is with respect to the parameter vector only, not the attribute values $x$).  

Regression problems - $y \in \mathbb R$
$$\frac12\sum_{x,y\,\in \, X,Y} (y - f(x, w))^2$$
$$\sum_{x,y\, \in\, X,Y} |(y - f(x, w))|$$

Classification problems - $y = -1\, or\, 1$
$$\frac{1}{ln\,2}\, \sum_{x,y\, \in\, X,Y} ln(1+e^{-yf(x,w)})$$
$$max(0, 1 - yf(x,w))$$

##Factorization Machines
The exercise above demonstrates how the gradient of the prediction function supplies you with what you need to calculate the gradient of many different performance measures.  That's what makes factorization machines able to tackle so many different types of problems.  Now look at the nature of the solutions that a factorization machine generates.  

$$\hat{y}(\textbf x) = w_0 + \sum_{i=1}^N w_i x_i + \sum_{i=1}^N \sum_{j=i+1}^n \langle v_i,v_j \rangle x_i x_j$$

Several elements of this equation require some elaboration.  The first two terms mirror the linear equation above.  The third terms is a quadratic term that picks up interactions between variables.  It assigns a weight to terms like $x_ix_j$.  Recall that this is also possible in linear regression by what is called basis expansion - where we would add new attributes to the data matrix.  The new attributes would be products of the existing attributes.  A row of the initial data might look like $[x_1,\, x_2,\, x_3]$ and the expanded data row like $[x_1,\, x_2,\, x_3,\, x_1x_2,\, x_1x_3,\, x_2x_3]$.  The process of fitting a linear model (regression, svm etc.) will find coefficient values (or weights) for the cross product terms that are added by basis expansion.  

That's what's happening in the third term of factorization machine equation above.  The inner products $\langle v_i,v_j \rangle$ are coefficients (or weights) for the cross product term $x_ix_j$.  They don't look like the usual weights because they're the result of taking an inner product of two vectors.  If you arrange these vectors as rows in a matrix V

$$V = \begin{bmatrix}
v_1\\
v_2\\
.\\
v_N\end{bmatrix}$$

If you alter the limits on the sum in the factorization machine equation then you can write the sum as a quadratic form.  

$$\sum_{i=1}^N \sum_{j=1}^N \langle v_i,v_j \rangle x_i x_j = \textbf x^T VV^T \textbf x$$

The matrix $VV^T$ is square symmetrix NxN matrix.  It is positive semi definite.  It may or may not be full rank depending on the rank of V.  So you can think of the matrix $VV^T$ as containing the the coefficients that will be applied to the cross products of various elements of $\textbf x$.  The sum above is a little different from the sum in the factorization machine definition.  In the sum above the limits of the sums cover all the elements of the matrix $VV^T$.  In the matrix factorization sums they only cover the upper triangle of the matrix (that is the elements above the diagonal).  Rendle shows in the proof of Lemma 3.1 that the sum over the upper triangle is equal to the sum over the lower triangle.  Therefore the sum over the whole matrix minus the sum over the diagonal is twice the sum over the upper triangle.  

##Question
1  Suppose that you have three matrices U (for upper), D (for diagonal) and L (for lower).  Where 
$$U(i,j) = VV^T(i,j)\, for\, j>i$$
$$U(i,j) = 0.0\, otherwise$$
$$D(i,j) = VV^T(i,j)\, for\, i=j$$
$$D(i,j) = 0.0\, otherwise$$
$$L(i,j) = VV^T(i,j)\, for\, j < i $$
$$L(i,j) = 0.0\, otherwise$$

Prove that $${\textbf x}^TU{\textbf x} + {\textbf x}^TD{\textbf x} +{\textbf x}^TL{\textbf x} = {\textbf x}^TVV^T{\textbf x}$$ and 
$${\textbf x}^TU{\textbf x} = {\textbf x}^TL{\textbf x}$$

##Comparisons between Factorization Machine and Other ML Algo
If two algorithms have the same functional form and optimize the same performance measure, then they have to give the same models.  You've already seen that the factorization machines can be used with the performance measures for SVM, linear regression, logistic regression, etc.  Since factorization machines include linear weighting of the features, that means that factorization machines can produce the same models as linear versions of the algorithms.  What about non-linear versions - say polynomial kernels for SVM?  The reference listed above (polynomial kernel) shows what a quadratic kernel looks like in terms of the feature mapping that's implicit in a second order kernel.  A little manipulation demonstrates that 2nd order factorization machine could generate the coefficient values for the cross-product terms in the polynomial kernel.  Factorization machine leaves out the diagonal ($x_i^2$) terms, but those are easy enough to add back in if you think they are important to your problem.  In a recommender problem the $x_i^2$ terms are not interesting because in the factorization machine formulation the user and the item attributes are all one-hot coded so that $x_i = 0\, or\, 1$ and therefore $x_i^2 = x_i$.  For other problems you may wish to add these terms into the formulation.  

A factorization machine will assign coefficients to all the cross product terms, but they are somewhat constrained compared by the nature of the factor model.

##Question
Suppose that the matrix V is NxK where N is the number of attributes and K < N.  Define matrix $W\,\triangleq\,VV^T$. What is the dimension of W?  What is the rank of W?  

Instead of having all possible values for the weights the (reduced rank) factor form imposes some constraints on the resulting weights.  Is that good or bad?  A thought example may help you decide.  

Consider a text classification problem where you're given a document-term matrix to use for classifying documents.  Word pairs may be useful for developing your classifier.  To separately estimate coefficients for all possible pairs of words, those words must appear together in a number of documents.  As you know from you NLP course, different words may be used to indicate the same concepts (physician, doctor, healer).  That and the fact that document-term matrices are usually extremely (95%) sparse means that you'll have a hard time estimating separate weights for all possible pairs of words.  Recall that latent semantic indexing uses singular value decomposition to develop a reduced rank approximation for the document-term matrix and by reducing the rank of the matrix develops an approximation which identifies themes or concepts that can be blended to accurately describe documents.  That's the role that the reduced-rank factor form plays.  So on a sparse data set you have good reason to suspect that a factorization machine may outperform methods that have more degrees of freedom.  

##Question
How can you test the assumption that reduced rank factor gives better performance than a full rank solution?  How do you go about choosing the best rank (K)?